In [ ]:
import pandas as pd
import glob
from datetime import datetime
from tqdm import tqdm
from geopy.extra.rate_limiter import RateLimiter
import time
import googlemaps

# Import bus data and streetcar data csv files

In [ ]:
# combine excel sheets together

df_bus = pd.read_csv('BusDelayData_updated.csv')
df_st = pd.read_csv('StreetcarDelayData.csv')

In [ ]:
# shape of the data

print(df_bus.shape)
print(df_st.shape)

In [ ]:
df_bus.describe()

### create column for categorizing bus and streetcar prior to merging datasets

In [ ]:
df_bus['vtype'] = 'bus'
df_st['vtype'] = 'streetcar'

In [ ]:
df_st.describe()

In [ ]:
# combine bus and streetcar
df_tot = pd.concat([df_bus, df_st])

In [ ]:
df_tot.shape

In [ ]:
df_tot.head(10)

In [ ]:
df_tot[df_tot['vtype'] == 'bus'].count() 

In [ ]:
df_tot.tail()

In [ ]:
df_tot = df_tot.drop(columns = 'Unnamed: 0')

In [ ]:
df_tot.info()

## convert report date to datetime format and create year, month and day columns
## convert time to time format to create hour and minute column

In [ ]:
df_tot['Report Date'] = pd.to_datetime(df_tot['Report Date'])

In [ ]:
df_tot.head(5)

In [ ]:
df_tot = df_tot.reset_index().drop(columns = 'index')

In [ ]:
df_tot.info()

In [ ]:
df_tot['year'] = pd.DatetimeIndex(df_tot['Report Date']).year
df_tot['month'] = pd.DatetimeIndex(df_tot['Report Date']).month
df_tot['day'] = pd.DatetimeIndex(df_tot['Report Date']).day

In [ ]:
import datetime as dt

df_tot['hour'] = pd.to_datetime(df_tot['Time']).dt.hour
df_tot['minute'] = pd.to_datetime(df_tot['Time']).dt.minute

In [ ]:
df_tot['Location'].str.lower()

In [ ]:
df_tot['Location'] = df_tot['Location'].str.lower()
df_tot['Incident'] = df_tot['Incident'].str.lower()
df_tot['Day'] = df_tot['Day'].str.lower()
df_tot['Direction'] = df_tot['Direction'].str.lower()

df_tot

## Remove null values
## remove rows 

In [ ]:
df_tot.isnull().sum()

In [ ]:
#df_tot.shape

In [ ]:
df_tot = df_tot.dropna()

In [ ]:
df_tot.isnull().sum()

In [ ]:
df_tot.shape

## Clean location data first prior to retreiving lat and long coordinates

In [ ]:
df_tot = df_tot.reset_index()

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('&', 'and')

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('stn', 'station')

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('ave', 'avenue')

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('str', 'street')

In [ ]:
df_tot['Day'] = df_tot['Day'].str.capitalize()

In [ ]:
df_tot['Direction'] = df_tot['Direction'].str.replace('/', '')

In [ ]:
df_tot = df_tot.drop(columns = 'index')

In [ ]:
df_tot['Vehicle'] = df_tot['Vehicle'].astype(int)

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('/', '')

In [ ]:
df_tot['Location'] = df_tot['Location'].str.replace('#', '')
df_tot['Location'] = df_tot['Location'].str.replace('!', '')

In [ ]:
df_tot.head(5)

In [ ]:
df_tot['Location'].value_counts()[:20]

In [ ]:
location = df_tot['Location'].unique().tolist()

len(location)

In [ ]:
address= 'howardpark and dundas' + ' Toronto'
loc = geolocator.geocode(address)
if loc != None:
    print(loc.latitude, loc.longitude)
else:
    print(0)

## Use geopy first to retrieve lat and long coordinates before using google api since we are limited in the number of coordinates we can retrieve.

In [ ]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist_total =[]
longlist_total =[]


for i in tqdm(location):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist_total.append(loc.latitude)
        longlist_total.append(loc.longitude)
    else:
        latlist_total.append(0)
        longlist_total.append(0)        

In [ ]:
print(len(latlist_total))
print(len(longlist_total))

In [ ]:
d = {'location': location, 'lat': latlist_total, 'long': longlist_total}

In [ ]:
# put lat and long data into a dataframe
df_location = pd.DataFrame(data = d)

df_location
print(len(df_location[df_location['lat'] != 0.000000]))
print(len(df_location[df_location['lat'] == 0.000000]))

#lat: 42
#long: 58

In [ ]:
df_location[:40000][df_location['lat'] == 0.000000]

In [ ]:
df_location.to_csv('location_coor.csv') # save coordinates retrieved using geopy

In [ ]:
latlist_g = df_location['lat'].tolist()
longlist_g = df_location['long'].tolist()

In [ ]:
len(location)

In [ ]:
len(latlist_g)

## Retrieve remaining missing lat and long coordinates using googlemaps API

In [ ]:
latlist_g2 = []
longlist_g2 = []
location_g2 = []


for i,j,k in tqdm(zip(latlist_g, longlist_g, location)):
    
    try: 
        if i == 0:
            
            address_c = k + ', Toronto,' + ' Canada'

            gmaps = googlemaps.Client(key='{insert api key here}')

            try: 
                geocode_result = gmaps.geocode(address_c)
                if geocode_result != []:
                    try:
                        lat = geocode_result[0]['geometry']['location']['lat']
                        #print(lat)
                        long = geocode_result[0]['geometry']['location']['lng']
                        #print(long)
                        latlist_g2.append(lat)
                        longlist_g2.append(long)
                        location_g2.append(k)
                    
                    except:
                        latlist_g2.append(i)
                        longlist_g2.append(j)
                        location_g2.append(k)   

                else: 
                    latlist_g2.append(i)
                    longlist_g2.append(j)
                    location_g2.append(k)      

            except:
                latlist_g2.append(i)
                longlist_g2.append(j)
                location_g2.append(k) 

        else:
            latlist_g2.append(i)
            longlist_g2.append(j)
            location_g2.append(k)

    except:
        latlist_g2.append(i)
        longlist_g2.append(j)
        location_g2.append(k) 
            
            

# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit

In [ ]:
print(len(latlist_g2))
print(len(longlist_g2))

In [ ]:
len(latlist_g)

In [ ]:
len(longlist_g)

In [ ]:
len(latlist_g2)

In [ ]:
len(longlist_g2)

In [ ]:
len(location)

In [ ]:
d = {'location': location, 'lat': latlist_g2, 'long': longlist_g2}

In [ ]:
df_location = pd.DataFrame(data = d)

df_location.set_index('location')

df_location = df_location.rename(columns = {'location': 'Location'})
df_location.set_index('Location')

In [ ]:
df_location.loc[39999]

In [ ]:
latlist_g2[39999]

## random test case for coordinates

In [ ]:
df_location[df_location['Location'] == 'sloane and sweeney']

In [ ]:
df_location[df_location['Location'] == 'exhibition place']

## Combine location data to bus and streetcar data using location common column

In [ ]:
df_tot_v1 = pd.merge(df_tot, df_location)

In [ ]:
df_tot_v1

In [ ]:
df_tot_v1.info()

In [ ]:
df_tot_v1.to_csv('BusandStreetcarDelayData_v2.csv')